In [ ]:
import numpy as np
import time
import json
import requests
import pandas as pd

# defining lists which will be used to parse the data

important_keys = ['date_added', 'status', 'year', 'num_for_sale', 'artists', 'data_quality', 'country', 'resource_url', 'lowest_price', 'labels', 'formats', 'genres', 'community', 'date_changed']
artists = ['resource_url', 'name']
labels = ['resource_url', 'name'] # check if labels are often more than 1
formats = ['name', 'descriptions']
community = ['contributors', ['username'], 'rating', ['average'], ['count'], 'want', 'submitter', ['username'], 'have']
l_l = ['date_added', 'status', 'year', 'num_for_sale', 'artists', 'data_quality', 'country', 'resource_url', 'lowest_price', 'labels', 'name_format', 'genres', 'username', 'average', 'count', 'want', 'submitter','have','date_changed']
sub_keys = ['artists', 'labels', 'formats']

# creating a dataframe to collect API calls that returned an error or no results

dfmissing = pd.DataFrame(columns = ['address'])

# creating a dictionary that will be used to fill in data from API calls that return an error or no results

exc2 = {'date_added': np.nan, 
        'status': np.nan, 
        'year': np.nan, 
        'num_for_sale': np.nan, 
        'artists': [np.nan], 
        'data_quality': np.nan, 
        'country': np.nan, 
        'resource_url': np.nan, 
        'lowest_price': np.nan,
        'labels': [np.nan], 
        'name_format': np.nan, 
        'genres': [np.nan], 
        'username': [np.nan], 
        'average': np.nan, 
        'count': np.nan, 
        'want': np.nan, 
        'submitter': np.nan,
        'have': np.nan,
        'date_changed': np.nan,
       }

# querying the csv file will username + contribution page # combinations. Should be updated based on userProfileCollection. 

dfpages = pd.DataFrame.from_csv('page_names.csv')

df = pd.DataFrame(columns = l_l) # create dataframe to which all intermediate dataframes will be concatenated to. 

for n in range(40000, 60001): 
    check = time.time()
    print(n, " start")
    df4bis = pd.DataFrame(columns = l_l) # create dataframe that will be concatenated to df. 
    url = dfpages.loc[n,'address'] #get url address from dfpages dataframe. 
    headers = {
                'Authorization': 'Discogs key=[key], secret=[secret]',
            }
    try:
        r = requests.get(url, headers = headers)
        store = json.loads(r.text) 
        print("-extraction-- %s seconds ---" % (time.time() - check)) # get time of API call. 
        if 'contributions' in store.keys(): #parsing of the data.
            for i in store['contributions']:
                l_global = []
                l_sub = []
                for j in important_keys:
                    if j in i.keys():
                        if j in i.keys():
                            if j == 'artists':
                                for k in range(0,len(i[j])):
                                    l1 = []
                                    for l in artists:
                                        l1.append({l: i[j][k][l]})
                                l_global.append({j: l1})
                            if j == 'labels':
                                for k in range(0,len(i[j])):
                                    l3 = []
                                    for l in labels:
                                        if l == 'resource_url':
                                            l3.append({'resource_url_label': i[j][k][l]})
                                        if l == 'name': 
                                            l3.append({'name_label': i[j][k][l]})
                                l_global.append({j: l3})
                            if j == 'formats':
                                for k in range(0,len(i[j])):
                                    for l in formats:
                                        l4 = []
                                        if l == 'name':
                                            l4.append({'name_format': i[j][k][l]})
                                l_global.append({j: l4})
                            if j == 'community':
                                for k in community:
                                    l2 = []
                                    if k == 'contributors':
                                        for l in range(0, len(i[j][k])):                    
                                            l2.append(i[j][k][l]['username'])
                                        l_global.append({'username': l2})
                                    if k == 'rating':
                                        for l in i[j][k]:
                                            l_global.append({l:i[j][k][l]})
                                    if k == 'want':
                                        l_global.append({k: i[j][k]})
                                    if k == 'have':
                                        l_global.append({k: i[j][k]})
                                    if k == 'submitter':
                                        try:
                                            l_global.append({k: i[j][k]['username']})
                                        except TypeError: 
                                            l_global.append({k: 'None'})
                            elif j not in sub_keys : 
                                l_global.append({j: i[j]})
                    elif j not in i.keys():
                        l_global.append({j: ''})
                for m in l_global:
                    l_sub.append(list(m.values())[0])
                df4bis.loc[len(df4bis)] = l_sub #adding parsed data to dataframe.
        else:
            dfmissing.loc[len(dfmissing)] = url #if error in API call, add the URL to a separate dataframe.
            print("url missing.")
    except (requests.ConnectionError, ValueError): 
        dfmissing.loc[len(dfmissing)] = url #if error in API call, add the URL to a separate dataframe.
        print('request timeout.')
    df = pd.concat([df, df4bis]) #add parsed dataframe to bigger dataframe
    print("-processing-- %s seconds ---" % (time.time() - check))
    if n%100==0:
        df.to_csv('%s.csv' % n) #every 100 calls, create csv from dataframe. 
        df = pd.DataFrame(columns = l_l)  #reinitiate dataframe df. 
    total_time = (time.time() - check)
    if total_time < 1: #if calls are too quick, add a pause time to the API calls (due to 60 calls per min. rate limitation).
        time.sleep(1-total_time)
        print("pause of", 1 - total_time)
dfmissing.to_csv('missing.csv') #create dataframe with missing URL adddesses. 